In [ ]:
import numpy as np
import pandas as pd
import pymc3 as pm
from scipy import stats
from sklearn.metrics import mean_squared_error
import theano
import random

import plotly.graph_objects as go

# 1

> The weights listed below were recorded in the !Kung census, but heights were not recorded for these individuals. Provide predicted heights and 89% compatibility intervals for each of these individuals. That is, fill in the table below, using model-based predictions.

In [ ]:
to_predict = [45, 40, 65, 31, 53]

First we want to load the data used to fit our model:

In [ ]:
howell = pd.read_csv('../../data/Howell1.csv', sep = ';')
howell.head()

Filter to only adults and standardize weight:

In [ ]:
howell_adults = howell[howell.age >= 18].copy()

In [ ]:
howell_adults['std_weight'] = howell_adults.weight - howell_adults.weight.mean()

Define and fit model:

In [ ]:
with pm.Model() as model_1:
    # Extract data
    weight = pm.Data('weight', howell_adults['std_weight'].values)
    height = pm.Data('height', howell_adults['height'].values)
    
    # Define priors
    alpha = pm.Normal('alpha', mu=178, sd=20)
    beta = pm.Normal('beta', mu=0, sd=10)
    sigma = pm.Uniform('sigma', lower=0, upper=50)
    
    # Define regression model
    mu = alpha + beta * weight
    height_hat = pm.Normal('height_hat', mu=mu, sd=sigma, observed=height)
    
    # Prior sampling, trace definition and posterior sampling
    prior = pm.sample_prior_predictive()
    posterior_1 = pm.sample(draws=1000, tune=1000)
    posterior_pred_1 = pm.sample_posterior_predictive(posterior_1)

In [ ]:
pm.summary(posterior_1, alpha=.10).round(2)

In [ ]:
pm.traceplot(posterior_1);

In the code above, after fitting the model we also used draws from the posterior to predict heights for each supplied weight point. We can average the predictions for each point to get a predictive point estimate, and assess our model's fit that way:

In [ ]:
height_hats = posterior_pred_1['height_hat'].mean(axis = 0)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = howell_adults['height'],
        y = height_hats,
        mode = 'markers'
    )
)


fig.update_layout(
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="height",
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="height_hat",
        )
    )
)

In [ ]:
np.sqrt(mean_squared_error(height_hats, howell_adults['height']))

Only 5cm - not bad!

We can also get the HPDI for each sample:

In [ ]:
pm.hpd(posterior_pred_1['height_hat'], alpha = .11)

Now we can predict for our out-of-sample weight points. We need to standardize them and add them as data to the model:

In [ ]:
posterior_pred_1['height_hat'].shape

In [ ]:
x_test = (np.array(to_predict) - howell.weight.mean())
weight.set_value(x_test)

In [ ]:
posterior_pred_1 = pm.sample_posterior_predictive(trace = posterior_1, samples = 500, model = model_1)

In [ ]:
test = pd.DataFrame(to_predict, columns = ['weight'])

In [ ]:
test['height_hat'] = posterior_pred_1['height_hat'].mean(axis = 0)

In [ ]:
test = test.join(
    pd.DataFrame(
        pm.hpd(posterior_pred_1['height_hat'], alpha = .11), 
        columns = ['hpd5.5', 'hpd95.5']
    )
)

test

# 2

> Model the relationship between height (cm) and the natural logarithm of weight (log-kg): `log(weight)`. Use the entire `Howell1` data frame, all 544 rows, adults and non-adults. Use any model type from Chapter 4 that you think useful: an ordinary linear regression, a polynomial or a spline. Plot the posterior predictions against the raw data.

In [ ]:
howell['log_std_weight'] = np.log(howell['weight']) - np.log(howell['weight'].mean())

In [ ]:
with pm.Model() as model_2:
    # Extract data
    log_weight = pm.Data('log_weight', howell['log_std_weight'].values)
    height = pm.Data('height', howell['height'].values)
    
    # Define priors
    alpha = pm.Normal('alpha', mu=178, sd=20)
    beta = pm.Normal('beta', mu=0, sd=10)
    sigma = pm.Uniform('sigma', lower=0, upper=50)
    
    # Define regression model
    mu = alpha + beta * log_weight
    height_hat = pm.Normal('height_hat', mu=mu, sd=sigma, observed=height)
    
    # Prior sampling, trace definition and posterior sampling
    prior = pm.sample_prior_predictive()
    posterior_2 = pm.sample(draws=1000, tune=1000)
    posterior_pred_2 = pm.sample_posterior_predictive(posterior_2)

In [ ]:
with pm.Model() as model_3:
    # Extract data
    log_weight = pm.Data('log_weight', howell['log_std_weight'].values)
    log_weight_sq = log_weight ** 2
    height = pm.Data('height', howell['height'].values)
    
    # Define priors
    alpha = pm.Normal('alpha', mu=178, sd=20)
    beta1= pm.Normal('beta1', mu=0, sd=10)
    beta2 = pm.Normal('beta2', mu = 0, sd = 1)
    sigma = pm.Uniform('sigma', lower=0, upper=50)
    
    # Define regression model
    mu = alpha + beta1 * log_weight + beta2 * log_weight_sq
    height_hat = pm.Normal('height_hat', mu=mu, sd=sigma, observed=height)
    
    # Prior sampling, trace definition and posterior sampling
    prior = pm.sample_prior_predictive()
    posterior_3 = pm.sample(draws=1000, tune=1000)
    posterior_pred_3 = pm.sample_posterior_predictive(posterior_3)

In [ ]:
fig = go.Figure()
mean_lin_reg = pd.Series(posterior_pred_2['height_hat'].mean(axis = 0))
hpdi_lin_reg = pm.hpd(posterior_pred_2['height_hat'], alpha = .11)
mean_polynomial =  pd.Series(posterior_pred_3['height_hat'].mean(axis = 0))
hpdi_polynomial = pm.hpd(posterior_pred_3['height_hat'], alpha = .11)

fig.add_trace(
    go.Scatter(
        x = howell['weight'],
        y = howell['height'],
        mode = 'markers',
        name = 'actual',
        opacity = .4
    )
)

fig.add_trace(
    go.Scatter(
        x = howell['weight'],
        y = mean_lin_reg,
        name = 'linear regression',
        mode ='markers'
    )
)

fig.add_trace(
    go.Scatter(
        x = howell['weight'],
        y = mean_polynomial,
        name = 'polynomial regression',
        mode = 'markers'
    )
)

fig.update_layout(
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="weight",
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="height"
        )
    )
)

Pretty similar results for the linear and polynomial regression attempts.

# 3

> Plot the prior predictive distribution for the polynomial regression model in Chapter 4. You can modify the code that plots the linear regression prior predictive distribution. 20 or 30 parabolas from the prior should suffice to show where the prior probability resides. Can you modify the prior distributions of $\alpha$, $\beta_1$, and $\beta_2$ so that the prior predictions stay within the biologically reasonable outcome space? That is to say: Do not try to fit the data by hand. But do try to keep the curves consistent with what you know about height and weight, before seeing these exact data.

To get the prior predictive distribution, we simply sample from the prior distributions for each parameter in our polynomial model, and  compute the predicted output variables using our data.

In [ ]:
n_samples = 20
alpha = np.random.normal(178, 20, n_samples)
beta_1 = np.random.lognormal(0, 1, n_samples)
beta_2 = np.random.normal(0, 1, n_samples)

In [ ]:
weights = howell['weight'] - howell['weight'].mean()
weights = np.repeat(weights.to_numpy().reshape(-1,1), n_samples, axis = 1)
height_hat = alpha + (beta_1 * weights) + (beta_2 * weights**2)

In [ ]:
fig = go.Figure()

for curve in range(weights.shape[1]):
    fig.add_trace(
        go.Scatter(
            x = howell['weight'].to_numpy(),
            y = height_hat[:, curve],
            mode = 'markers'
        )
    )
fig.update_layout(
    showlegend = False,
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="weight",
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="height"
        )
    )
)
fig.show()     